# AutoQuant

This notebook shows a working code example of how to use AIMET AutoQuant feature.

AIMET offers a suite of neural network post-training quantization (PTQ) techniques that can be applied in succession. However, the process of finding the right combination and sequence of techniques to apply is time-consuming and requires careful analysis, which can be challenging especially for non-expert users. We instead recommend AutoQuant to save time and effort.

AutoQuant is an API that applies various PTQ techniques in AIMET automatically based on analyzing the model and best-known heuristics. In AutoQuant, users specify the amount of tolerable accuracy drop, and AutoQuant will apply PTQ techniques cumulatively until the target accuracy is satisfied.


#### Overall flow
This notebook covers the following
1. Define constants and helper functions
2. Load a pretrained FP32 model
3. Run AutoQuant

#### What this notebook is not
This notebook is not designed to show state-of-the-art AutoQuant results. For example, it uses a relatively quantization-friendly model like Resnet18. Also, some optimization parameters are deliberately chosen to have the notebook execute more quickly.


---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification. If you already have a version of the dataset readily available, please use that. Else, please download the dataset from appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#).

**Note1**: The ImageNet dataset typically has the following characteristics and the dataloader provided in this example notebook rely on these
- Subfolders 'train' for the training samples and 'val' for the validation samples. Please see the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- A subdirectory per class, and a file per each image sample

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset. E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class. But for the purpose of running this notebook, you could perhaps reduce the dataset to say 2 samples per class. This exercise is left upto the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [1]:
import cv2
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import json
from mmcv.transforms import Compose
import numpy as np
from mmdet.utils import get_test_pipeline_cfg

id_to_keep = 179487

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)
        self.annotations_json['images'] = [x for x in self.annotations_json['images'] if int(x['id']) == int(id_to_keep)] + self.annotations_json['images'][:9]


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)

        image_id = image_dict['id']
        
        return image_id, image

# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

## 1. Define Constants and Helper functions

In this section the constants and helper functions needed to run this eaxmple are defined.

- **EVAL_DATASET_SIZE** A typical value is 5000. In this notebook, this value has been set to 500 for faster execution.
- **CALIBRATION_DATASET_SIZE** A typical value is 2000. In this notebook, this value has been set to 200 for faster execution.


The helper function **_create_sampled_data_loader()** returns a DataLoader based on the dataset and the number of samples provided.

## 2. Load a pretrained FP32 model
For this example, we are going to load a pretrained resnet18 model from torchvision. Similarly, you can load any pretrained PyTorch model instead.

In [2]:
# %cd /content/drive/MyDrive/Aimet-torch/mmdetection-3.3.0/
import torch
from mmdet.apis import init_detector

transform = transforms.Compose([
            transforms.Resize([640, 640]),  # Resize
        ])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = 'rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = 'rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

ROOT_DATASET_DIR = '/datasets/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')

model = init_detector(CONFIG_PATH, WEIGHTS_PATH, device=DEVICE)
predict_by_feat = model.bbox_head.predict_by_feat
calibrationDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH)
test_pipeline = get_test_pipeline_cfg(model.cfg)

2024-07-30 16:28:05,044 - root - INFO - AIMET
Loads checkpoint by local backend from path: rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



In [3]:
from mmcv.transforms import Compose
test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator = Compose(test_evaluator)

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


In [4]:
import mmcv
import mmengine
import os.path as osp

from mmengine.registry import FUNCTIONS, MODELS, VISUALIZERS, DefaultScope
from mmengine.visualization import Visualizer
from datetime import datetime


def init_visualizer(cfg):
    """Initialize visualizers.

    Args:
        cfg (ConfigType): Config containing the visualizer information.

    Returns:
        Visualizer or None: Visualizer initialized with config.
    """
    if 'visualizer' not in cfg:
        return None
    timestamp = str(datetime.timestamp(datetime.now()))
    name = cfg.visualizer.get('name', timestamp)
    if Visualizer.check_instance_created(name):
        name = f'{name}-{timestamp}'
    cfg.visualizer.name = name
    return VISUALIZERS.build(cfg.visualizer)

visualizer = init_visualizer(model.cfg)
visualizer.dataset_meta = model.dataset_meta

def visualize(inputs,
                  preds,
                  return_vis = False,
                  show = False,
                  wait_time = 0,
                  draw_pred = True,
                  pred_score_thr = 0.3,
                  no_save_vis = False,
                  img_out_dir = '/home/shayaan/Desktop/aimet/my_mmdet/pred_results',
                  img_id = None,
                  **kwargs):
        """Visualize predictions.

        Args:
            inputs (List[Union[str, np.ndarray]]): Inputs for the inferencer.
            preds (List[:obj:`DetDataSample`]): Predictions of the model.
            return_vis (bool): Whether to return the visualization result.
                Defaults to False.
            show (bool): Whether to display the image in a popup window.
                Defaults to False.
            wait_time (float): The interval of show (s). Defaults to 0.
            draw_pred (bool): Whether to draw predicted bounding boxes.
                Defaults to True.
            pred_score_thr (float): Minimum score of bboxes to draw.
                Defaults to 0.3.
            no_save_vis (bool): Whether to force not to save prediction
                vis results. Defaults to False.
            img_out_dir (str): Output directory of visualization results.
                If left as empty, no file will be saved. Defaults to ''.

        Returns:
            List[np.ndarray] or None: Returns visualization results only if
            applicable.
        """
        if no_save_vis is True:
            img_out_dir = ''

        if not show and img_out_dir == '' and not return_vis:
            return None


        results = []
        num_visualized_imgs = 0

        for single_input, pred in zip(inputs, preds):
            if isinstance(single_input, str):
                img_bytes = mmengine.fileio.get(single_input)
                img = mmcv.imfrombytes(img_bytes)
                img = img[:, :, ::-1]
                img_name = osp.basename(single_input)
            elif isinstance(single_input, np.ndarray):
                img = single_input.copy()
                img_num = str(num_visualized_imgs).zfill(8)
                img_name = f'{img_num}.jpg'
            else:
                raise ValueError('Unsupported input type: '
                                 f'{type(single_input)}')
            if img_id:
                img_name = f'{img_id}.jpg'
            out_file = osp.join(img_out_dir, 'vis',
                                img_name) if img_out_dir != '' else None

            visualizer.add_datasample(
                img_name,
                img,
                pred,
                show=show,
                wait_time=wait_time,
                draw_gt=False,
                draw_pred=draw_pred,
                pred_score_thr=pred_score_thr,
                out_file=out_file,
            )
            results.append(visualizer.get_image())
            num_visualized_imgs += 1

        return results


/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [5]:
import random
from typing import Optional
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, Subset
from aimet_torch.utils import in_eval_mode, get_device
from mmdet.apis import inference_detector
from mmengine.structures import InstanceData
from torchsummary import summary

EVAL_DATASET_SIZE = 500
CALIBRATION_DATASET_SIZE = 200
BATCH_SIZE = 1

_datasets = {}

def _create_sampled_data_loader(dataset, num_samples):
    if num_samples not in _datasets:
        indices = random.sample(range(len(dataset)), num_samples)
        _datasets[num_samples] = Subset(dataset, indices)
    return DataLoader(_datasets[num_samples], batch_size=BATCH_SIZE)


def eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    if num_samples is None:
        num_samples = EVAL_DATASET_SIZE

    data_loader = _create_sampled_data_loader(calibrationDataset, 10)
    device = get_device(model)

    correct = 0
    eval_metrics = []
    for image_id, image in tqdm(data_loader):
        if int(image_id) != id_to_keep:
            continue
        numpy_image = image.numpy()[0]
        image_dict = preprocess(test_pipeline, numpy_image)
        data_samples = image_dict['data_samples']
        print(type(image_dict['inputs']))
        image = image_dict['inputs'].reshape(1, 3, 640, 640)
        
        print(image.dtype)
        preds = model(image)

        for x in preds:
            for t in x:
                t = t.detach().numpy()
                name = "_".join([str(x) for x in t.shape])
                path = "/home/shayaan/Desktop/aimet/my_mmdet/pred_results/new_raw_results/" + str(int(image_id)) + "_" + name
                np.save(path, t)
        
        batch_img_metas = [data_samples.metainfo]
        predictions = predict_by_feat(
            *preds, batch_img_metas=batch_img_metas)
        
        print("Predictions for image_id ----->>", int(image_id))

        res = visualize([numpy_image], predictions, img_id=int(image_id))
            

        new_pred = InstanceData(metainfo={"img_id": int(image_id)})
        new_pred.bboxes = predictions[0].bboxes
        new_pred.labels = predictions[0].labels
        new_pred.scores = predictions[0].scores
        new_pred = [new_pred]
        eval_results = test_evaluator(new_pred)
        eval_metrics.append(eval_results)
        break
    
    bbox_map = sum([metric['bbox_mAP'] for metric in eval_metrics]) / len(eval_metrics)
    return bbox_map

In [6]:
# from aimet_torch.quantsim import QuantizationSimModel, QuantScheme
# from aimet_torch.model_preparer import prepare_model

# # quant_model = QuantizationSimModel(model, torch.rand((1, 3, 224, 224)))

# dummy_input = torch.rand(1, 3, 640, 640)  # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)
# # # if use_cuda:
# # #     dummy_input = dummy_input.cuda()

# print("constructor called for Quant Sim Model")

# sim_model = prepare_model(model.model)

# sim = QuantizationSimModel(model=sim_model,
#                         quant_scheme=QuantScheme.post_training_tf_enhanced,
#                         dummy_input=dummy_input,
#                         default_output_bw=8,
#                         default_param_bw=8,)

In [8]:
accuracy = eval_callback(model)

print(f'- FP32 accuracy: {accuracy}')

 10%|█         | 1/10 [00:00<00:00, 48.81it/s]

<class 'torch.Tensor'>
torch.uint8


RuntimeError: expected scalar type Byte but found Float

In [111]:
# from torchvision.models import resnet18

# model = resnet18(pretrained=True).eval()


## 3. Run AutoQuant
### Create AutoQuant Object

The AutoQuant feature utilizes an unlabeled dataset to achieve quantization. The class **UnlabeledDatasetWrapper** creates an unlabeled Dataset object from a labeled Dataset.

In [1]:
transform = Compose([
    transforms.ToTensor(),
    transforms.Resize((640, 640)),
])

class UnlabeledDatasetWrapper(Dataset):
    def __init__(self, dataset, transform=None):
        self._dataset = list(dataset)[:10]

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, index):
        image_id, image_path = self._dataset[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)

        transformed_images = transform(image)
        return image_id, image_path, transformed_images
    
# next(iter(DataLoader(UnlabeledDatasetWrapper(calibrationDataset), 10)))
# next(iter(UnlabeledDatasetWrapper(calibrationDataset)))

NameError: name 'Compose' is not defined

In [156]:
from aimet_torch.auto_quant import AutoQuant



data_loader = DataLoader(UnlabeledDatasetWrapper(calibrationDataset))
def new_eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    if num_samples is None:
        num_samples = EVAL_DATASET_SIZE

    device = get_device(model)

    correct = 0
    eval_metrics = []
    for image_id, _, image in tqdm(data_loader):
        image = image.numpy()[0]
        image_dict = preprocess(test_pipeline, image)
        data_samples = image_dict['data_samples']
        image = image_dict['inputs'].reshape(1, 3, 640, 640).float()
        preds = model(image)
        batch_img_metas = [data_samples.metainfo]
        predictions = predict_by_feat(
            *preds, batch_img_metas=batch_img_metas)

        new_pred = InstanceData(metainfo={"img_id": int(image_id)})
        new_pred.bboxes = predictions[0].bboxes
        new_pred.labels = predictions[0].labels
        new_pred.scores = predictions[0].scores
        new_pred = [new_pred]
        eval_results = test_evaluator(new_pred)
        eval_metrics.append(eval_results)

    return correct / num_samples


auto_quant = AutoQuant(sim_model,
                       dummy_input=dummy_input,
                       data_loader=data_loader,
                       eval_callback=new_eval_callback)

### Run AutoQuant Inference
This step runs AutoQuant inference. AutoQuant inference will run evaluation using the **eval_callback** with the vanilla quantized model without applying PTQ techniques. This will be useful for measuring the baseline evaluation score before running AutoQuant optimization.

In [157]:
sim, initial_accuracy = auto_quant.run_inference()
print(f"- Quantized A ccuracy (before optimization): {initial_accuracy}")

2024-07-29 13:51:42,374 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm} 


2024-07-29 13:51:42,376 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul} 


2024-07-29 13:51:42,377 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_1} 


2024-07-29 13:51:42,378 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul_1} 


2024-07-29 13:51:42,379 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_2} 


...

2024-07-29 13:51:44,815 - Utils - INFO - All validation checks passed.


2024-07-29 13:51:44,817 - AutoQuant - INFO - Model validation has succeeded. Proceeding to AutoQuant algorithm.


- Prepare Model
/ Batchnorm Folding


Traceback (most recent call last):
  File "/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/aimet_torch/meta/connectedgraph.py", line 1324, in _find_aten_nodes_in_forward_pass
    nodes = [node for node in trace.graph.nodes() if "aten::" in node.kind() and
  File "/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/torch/jit/_script.py", line 705, in graph
    return self._c._get_method("forward").graph
RuntimeError: Method 'forward' is not defined.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/aimet_torch/auto_quant.py", line 333, in run_inference
    model, _ = sess.wrap(self._apply_batchnorm_folding)(model)
  File "/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/aimet_torch/auto_quant.py", line 1048, in wrapper
    ret = fn(*args, **kwargs)
  File "/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site

KeyboardInterrupt: 

### Set AdaRound Parameters (optional)
AutoQuant uses a set of predefined default parameters for AdaRound.
These values were determined empirically and work well with the common models.
However, if necessary, you can also use your custom parameters for Adaround.
In this notebook, we will use very small AdaRound parameters for faster execution.

In [11]:
from aimet_torch.adaround.adaround_weight import AdaroundParameters

adaround_data_loader = DataLoader(UnlabeledDatasetWrapper(calibrationDataset))
adaround_params = AdaroundParameters(adaround_data_loader, num_batches=len(adaround_data_loader), default_num_iterations=2000)
# auto_quant.set_adaround_params(adaround_params)
adaround_params.warm_start

0.2

### Run AutoQuant Optimization
This step runs AutoQuant optimization, which returns the best possible quantized model, corresponding evaluation score and the path to the encoding file.
The **allowed_accuracy_drop** parameter indicates the tolerable amount of accuracy drop. AutoQuant applies a series of quantization features until the target accuracy (FP32 accuracy - allowed accuracy drop) is satisfied. When the target accuracy is reached, AutoQuant will return immediately without applying furhter PTQ techniques. Please refer AutoQuant User Guide and API documentation for complete details.

In [12]:
model, optimized_accuracy, encoding_path = auto_quant.optimize(allowed_accuracy_drop=0.01)
print(f"- Quantized Accuracy (after optimization):  {optimized_accuracy}")

2024-07-29 10:18:56,679 - AutoQuant - INFO - Starting AutoQuant


  0%|          | 0/10 [00:00<?, ?it/s]

07/29 10:18:56 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.28s).
Accumulating evaluation results...


 10%|█         | 1/10 [00:04<00:36,  4.08s/it]

DONE (t=0.53s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
07/29 10:19:00 - mmengin

...

DONE (t=0.55s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
2024-07-29 10:26:48,714 

\ W32 Evaluation


2024-07-29 10:26:49,357 - AutoQuant - INFO - Session finished: Batchnorm Folding. (eval score: 0.000000)


- Batchnorm Folding


2024-07-29 10:26:50,066 - AutoQuant - INFO - Best eval score: 0.000000
- Quantized Accuracy (after optimization):  0.0


In [13]:
auto_quant.dummy_input.shape

from aimet_torch.quantsim import QuantizationSimModel

model = QuantizationSimModel(model, auto_quant.dummy_input)

2024-07-29 10:26:52,602 - Quant - INFO - No config file provided, defaulting to config file at /home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2024-07-29 10:26:52,618 - Quant - INFO - Unsupported op type Squeeze
2024-07-29 10:26:52,618 - Quant - INFO - Unsupported op type Mean
2024-07-29 10:26:52,625 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


---
## Summary

Hope this notebook was useful for you to understand how to use AIMET AutoQuant feature.

Few additional resources
- Refer to the AIMET API docs to know more details of the APIs and parameters
- Refer to the other example notebooks to understand how to use AIMET CLE and AdaRound features in a standalone fashion.